In [1]:
import pandas as pd
import requests
from sklearn import preprocessing 


In [2]:
start = 50
end = 100

In [3]:
pd.set_option('display.max_rows', 500)


In [4]:
match_summary_df = pd.read_csv('../data/encoded_match_data.csv')

In [5]:
match_summary_df = match_summary_df[match_summary_df['league'] == 1][start:end]
len(match_summary_df)

37

In [6]:
match_summary_df.columns

Index(['league', 'format', 'game_in_format', 'red_team_outcome',
       'blue_team_outcome', 'season_format', 'riot_match_id', 'min_start_time',
       'month', 'day', 'year', 'red_team_id', 'blue_team_id', 'winner_id',
       'loser_id'],
      dtype='object')

In [7]:
match_summary_df.head(10)

,league,format,game_in_format,red_team_outcome,blue_team_outcome,season_format,riot_match_id,min_start_time,month,day,year,red_team_id,blue_team_id,winner_id,loser_id
292,1,0,1,0,1,1,111504634866275320,2024-03-02 22:00:00+00:00,3,2,2024,2151,2147,2147,2151
293,1,0,1,0,1,1,111504634866275332,2024-02-11 22:00:00+00:00,2,11,2024,2403,2149,2149,2403
294,1,0,1,0,1,1,111504634866209770,2024-02-11 21:00:00+00:00,2,11,2024,2147,2146,2146,2147
295,1,0,1,0,1,1,111504634866275336,2024-02-12 00:00:00+00:00,2,12,2024,2153,2151,2151,2153
296,1,0,1,0,1,1,111504634866209768,2024-02-11 23:00:00+00:00,2,11,2024,2155,2154,2154,2155
297,1,0,1,1,0,1,111504634866275330,2024-02-10 22:00:00+00:00,2,10,2024,2149,2153,2149,2153
298,1,0,1,1,0,1,111504634866275324,2024-02-10 21:00:00+00:00,2,10,2024,2147,2403,2147,2403
299,1,0,1,0,1,1,111504634866275316,2024-02-10 23:00:00+00:00,2,10,2024,2154,2146,2146,2154
300,1,0,1,0,1,1,111504634866209762,2024-02-11 00:00:00+00:00,2,11,2024,2151,2155,2155,2151
301,1,0,1,1,0,1,111504634866209750,2024-02-05 00:00:00+00:00,2,5,2024,2153,2403,2153,2403


In [8]:
columns=match_summary_df.columns.to_list()
columns.extend(['patch_version'])

for clr in ['blue','red']:
    columns.append(F"{clr}_team_id")
    for pos in ['top','jungle','mid','bot','supp']:
        col = F"{clr}_{pos}_player"
        columns.append(col)
        col = F"{clr}_{pos}_champ"
        columns.append(col)

columns.extend(['rfc460Timestamp'])
for clr in ['blue','red']:
    for info in ['total_gold','inhibitors','towers','barons','total_kills','ocean_drakes','cloud_drakes','mountain_drakes','chemtech_drakes','infernal_drakes','hextech_drakes','elder_drakes']:
        col = F"{clr}_team_{info}"
        columns.append(col)

    for pos in ['top','jungle','mid','bot','supp']:
        for info in ['total_gold','level','kills','deaths','assists','creep_score','current_health','max_health']:
            col = F"{clr}_{pos}_{info}"
            columns.append(col)




In [9]:
columns

['league',
 'format',
 'game_in_format',
 'red_team_outcome',
 'blue_team_outcome',
 'season_format',
 'riot_match_id',
 'min_start_time',
 'month',
 'day',
 'year',
 'red_team_id',
 'blue_team_id',
 'winner_id',
 'loser_id',
 'patch_version',
 'blue_team_id',
 'blue_top_player',
 'blue_top_champ',
 'blue_jungle_player',
 'blue_jungle_champ',
 'blue_mid_player',
 'blue_mid_champ',
 'blue_bot_player',
 'blue_bot_champ',
 'blue_supp_player',
 'blue_supp_champ',
 'red_team_id',
 'red_top_player',
 'red_top_champ',
 'red_jungle_player',
 'red_jungle_champ',
 'red_mid_player',
 'red_mid_champ',
 'red_bot_player',
 'red_bot_champ',
 'red_supp_player',
 'red_supp_champ',
 'rfc460Timestamp',
 'blue_team_total_gold',
 'blue_team_inhibitors',
 'blue_team_towers',
 'blue_team_barons',
 'blue_team_total_kills',
 'blue_team_ocean_drakes',
 'blue_team_cloud_drakes',
 'blue_team_mountain_drakes',
 'blue_team_chemtech_drakes',
 'blue_team_infernal_drakes',
 'blue_team_hextech_drakes',
 'blue_team_elde

In [10]:
match_data_df = pd.DataFrame(columns=columns)

In [11]:
match_data_df

,league,format,game_in_format,red_team_outcome,blue_team_outcome,season_format,riot_match_id,min_start_time,month,day,...,red_bot_current_health,red_bot_max_health,red_supp_total_gold,red_supp_level,red_supp_kills,red_supp_deaths,red_supp_assists,red_supp_creep_score,red_supp_current_health,red_supp_max_health


In [12]:
def extract_fixed_info(json_data):
    ret_data = dict()
    ret_data['patch_version'] = json_data['gameMetadata']['patchVersion']
    ret_data['blue_team_id'] = json_data['gameMetadata']['blueTeamMetadata']['esportsTeamId']
    for i,pos in enumerate(['top','jungle','mid','bot','supp']):

        ret_data[F'blue_{pos}_player'] = json_data['gameMetadata']['blueTeamMetadata']['participantMetadata'][i]['esportsPlayerId']
        ret_data[F'blue_{pos}_champ'] = json_data['gameMetadata']['blueTeamMetadata']['participantMetadata'][i]['championId']

    ret_data['red_team_id'] = json_data['gameMetadata']['redTeamMetadata']['esportsTeamId']
    for i,pos in enumerate(['top','jungle','mid','bot','supp']):

        ret_data[F'red_{pos}_player'] = json_data['gameMetadata']['redTeamMetadata']['participantMetadata'][i]['esportsPlayerId']
        ret_data[F'red_{pos}_champ'] = json_data['gameMetadata']['redTeamMetadata']['participantMetadata'][i]['championId']

    return ret_data

In [13]:
def encode_dragons(team,info):
    ret_data = dict()
    for drake in ['ocean_drakes','cloud_drakes','mountain_drakes','chemtech_drakes','infernal_drakes','hextech_drakes','elder_drakes']:
        ret_data[F"{team}_{drake}"] = 0
    for d in info:
        k = F"{team}_{d}_drakes"
        ret_data[k] = ret_data[k] + 1
    
    return ret_data

In [14]:
def extract_changing_info(json_data):
    ret_data = dict()
    # ret_data['rfc460Timestamp'] = json_data['rfc460Timestamp']
    for clr in ['blue','red']:
        ret_data[F'{clr}_team_total_gold'] = json_data[F'{clr}Team']['totalGold']
        ret_data[F'{clr}_team_inhibitors'] = json_data[F'{clr}Team']['inhibitors']
        ret_data[F'{clr}_team_towers'] = json_data[F'{clr}Team']['towers']
        ret_data[F'{clr}_team_barons'] = json_data[F'{clr}Team']['barons']
        ret_data[F'{clr}_team_total_kills'] = json_data[F'{clr}Team']['totalKills']
        ret_data.update(encode_dragons(F'{clr}_team',json_data[F'{clr}Team']['dragons']))

        for i,pos in enumerate(['top','jungle','mid','bot','supp']):
            ret_data[F'{clr}_{pos}_total_gold'] = json_data[F'{clr}Team']['participants'][i]['totalGold']
            ret_data[F'{clr}_{pos}_level'] = json_data[F'{clr}Team']['participants'][i]['level']
            ret_data[F'{clr}_{pos}_kills'] = json_data[F'{clr}Team']['participants'][i]['kills']
            ret_data[F'{clr}_{pos}_deaths'] = json_data[F'{clr}Team']['participants'][i]['deaths']
            ret_data[F'{clr}_{pos}_assists'] = json_data[F'{clr}Team']['participants'][i]['assists']
            ret_data[F'{clr}_{pos}_creep_score'] = json_data[F'{clr}Team']['participants'][i]['creepScore']
            ret_data[F'{clr}_{pos}_current_health'] = json_data[F'{clr}Team']['participants'][i]['currentHealth']
            ret_data[F'{clr}_{pos}_max_health'] = json_data[F'{clr}Team']['participants'][i]['maxHealth']


    return ret_data


In [15]:
BASE_URL = 'https://feed.lolesports.com/livestats/v1/window/{}?startingTime={}'

In [16]:
def format_time(t,time_delta):
    conv_time = pd.to_datetime(t) + pd.to_timedelta(time_delta, unit='s')
    conv_time = conv_time.round('10s')
    return "{}T{}.000Z".format(str(conv_time.date()),str(conv_time.time()))

In [17]:
time_delta = 60
temp_list = []

for index, row in match_summary_df.iterrows():
    print(index)
    print(row)
    game_state = "in_game"
    fixed_info = None
    time_elapsed = 0

    while game_state == "in_game":

        url = BASE_URL.format(row['riot_match_id'],format_time(row['min_start_time'],time_elapsed))
        raw_data = requests.get(url)

        time_elapsed = time_elapsed + time_delta
        if raw_data.status_code == 204: # No content returned
            continue

        data = raw_data.json()
        if fixed_info == None:
            fixed_info = extract_fixed_info(data)

        prev_frame = data['frames'][0]
        for frame in data['frames'][1:]:
            delta_info = extract_changing_info(frame)
            if delta_info == prev_frame:
                continue
            delta_info.update(fixed_info)
            delta_info['rfc460Timestamp'] = frame['rfc460Timestamp']
            delta_info.update(row)
            temp_list.append(delta_info)
            prev_frame = delta_info


        game_state = data['frames'][-1]['gameState']


292
league                                       1
format                                       0
game_in_format                               1
red_team_outcome                             0
blue_team_outcome                            1
season_format                                1
riot_match_id               111504634866275320
min_start_time       2024-03-02 22:00:00+00:00
month                                        3
day                                          2
year                                      2024
red_team_id                               2151
blue_team_id                              2147
winner_id                                 2147
loser_id                                  2151
Name: 292, dtype: object
293
league                                       1
format                                       0
game_in_format                               1
red_team_outcome                             0
blue_team_outcome                            1
season_format              

In [18]:
match_data_df = pd.DataFrame.from_dict(temp_list)

In [19]:
match_data_df.head()

,blue_team_total_gold,blue_team_inhibitors,blue_team_towers,blue_team_barons,blue_team_total_kills,blue_team_ocean_drakes,blue_team_cloud_drakes,blue_team_mountain_drakes,blue_team_chemtech_drakes,blue_team_infernal_drakes,...,red_team_outcome,blue_team_outcome,season_format,riot_match_id,min_start_time,month,day,year,winner_id,loser_id
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,111504634866275320,2024-03-02 22:00:00+00:00,3,2,2024,2147,2151
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,111504634866275320,2024-03-02 22:00:00+00:00,3,2,2024,2147,2151
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,111504634866275320,2024-03-02 22:00:00+00:00,3,2,2024,2147,2151
3,0,0,0,0,0,0,0,0,0,0,...,0,1,1,111504634866275320,2024-03-02 22:00:00+00:00,3,2,2024,2147,2151
4,0,0,0,0,0,0,0,0,0,0,...,0,1,1,111504634866275320,2024-03-02 22:00:00+00:00,3,2,2024,2147,2151


In [20]:
match_data_df.to_csv(F'../data/match_time_data_LCS_{start}_{end}.csv',index=False)